In [6]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor
from typing import Union, List, Tuple, Any
# from tabulate import tabulate

class AuditLog:
    
    def __init__(self, columns: Union[List[str], Tuple[str]], WS_ID: str, TABLE_NAME_to_check:str, AUDIT_TABLE_NAME:str, LH_ID_to_check: str, LH_ID_audit: str = None, schema: str = None):
        '''
        - if `LH_ID_audit` is not given, it is  LH_ID_to_check automatically, i.e. audit table is in the same lakehouse as that of
        - if using lakehouse with Schema, please provide `schema` parameter
        '''
        self.WS_ID = WS_ID
        self.TABLE_NAME_to_check = TABLE_NAME_to_check
        self.AUDIT_TABLE_NAME = AUDIT_TABLE_NAME
        self.LH_ID_to_check = LH_ID_to_check
        self.LH_ID_audit = LH_ID_audit if LH_ID_audit else LH_ID_to_check
        self.schema = schema
        self.fixColumns = {'STARTTIME','ENDTIME','AUDITKEY','STATUS_ACTIVITY'}
        self.columns = tuple(set(columns).union(self.fixColumns))
        
        if self.schema:    
            self.PATH_TO_AUDIT_TABLE = f'abfss://{self.WS_ID}@onelake.dfs.fabric.microsoft.com/{self.LH_ID_audit}/Tables/{self.schema}/{self.AUDIT_TABLE_NAME}'
            self.PATH_TO_CHECKED_TABLE = f'abfss://{self.WS_ID}@onelake.dfs.fabric.microsoft.com/{self.LH_ID_to_check}/Tables/{self.schema}/{self.TABLE_NAME_to_check}'
        else:
            self.PATH_TO_AUDIT_TABLE = f'abfss://{self.WS_ID}@onelake.dfs.fabric.microsoft.com/{self.LH_ID_audit}/Tables/{self.AUDIT_TABLE_NAME}'
            self.PATH_TO_CHECKED_TABLE = f'abfss://{self.WS_ID}@onelake.dfs.fabric.microsoft.com/{self.LH_ID_to_check}/Tables/{self.TABLE_NAME_to_check}'
    
        self.log = {column: None for column in self.columns}
        self.log['STARTTIME'] = datetime.now() + timedelta(hours=7)
        self.log['STATUS_ACTIVITY'] = 'Not start'
        
    def initialDetail(self, initConfig: dict[str, Any]):
        assert set(initConfig.keys()).issubset(set(self.columns).difference()), f'initConfig must have the columns in {self.columns}'
        for column in initConfig:
            self.log[column] = initConfig[column]
        
    def __str__(self):
        return str(self.log)
    
class AuditLog_SPC(AuditLog):
    
    def __init__(self, WS_ID: str, TABLE_NAME_to_check:str, AUDIT_TABLE_NAME:str, LH_ID_to_check: str, LH_ID_audit: str = None, schema: str = None):
        '''
        - if `LH_ID_audit` is not given, it is  LH_ID_to_check automatically, i.e. audit table is in the same lakehouse as that of
        - if using lakehouse with Schema, please provide `schema` parameter
        '''
        super().__init__(['PIPELINENAME', 'PIPELINERUNID', 'TRIGGERTYPE', 'TABLE_NAME', 'FUNCTION_NAME','COUNTROWSBEFORE', 'COUNTROWSAFTER', 'ERRORCODE', 'ERRORMESSAGE'] ,WS_ID, TABLE_NAME_to_check, AUDIT_TABLE_NAME, LH_ID_to_check, LH_ID_audit, schema)

    def initialDetail(self,  pipelineName: str, pipelineId, TriggerType, TableName, functionName, ):
        super().initialDetail({'PIPELINENAME': pipelineName, 'PIPELINERUNID': pipelineId, 'TRIGGERTYPE': TriggerType, 'TABLE_NAME': TableName, 'FUNCTION_NAME': functionName})

In [ ]:
ad = AuditLog_SPC('SPC_UAT', 'factTest', 'auditTable', 'SilverLH', 'AuditLH')
ad.initialDetail('testPL', '123', 'manual', 'factTest', 'testFunction')

{'STATUS_ACTIVITY': 'Not start', 'TRIGGERTYPE': None, 'ERRORCODE': None, 'STARTTIME': datetime.datetime(2025, 1, 18, 4, 50, 58, 114075), 'ENDTIME': None, 'PIPELINERUNID': None, 'TABLE_NAME': None, 'ERRORMESSAGE': None, 'PIPELINENAME': None, 'AUDITKEY': None, 'COUNTROWSBEFORE': None, 'COUNTROWSAFTER': None, 'FUNCTION_NAME': None}


In [14]:
class FrozenKeysDict:
    def __init__(self, **kwargs):
        self._data = kwargs

    def __getitem__(self, key):
        return self._data[key]

    def __setitem__(self, key, value):
        if key not in self._data:
            raise KeyError(f"Cannot add new key: {key}")
        self._data[key] = value

    def __delitem__(self, key):
        raise KeyError(f"Cannot delete key: {key}")

    def __iter__(self):
        return iter(self._data)

    def __len__(self):
        return len(self._data)

    def __repr__(self):
        return repr(self._data)

In [18]:
log = FrozenKeysDict(**{column: None for column in ['PIPELINENAME', 'PIPELINERUNID', 'TRIGGERTYPE', 'TABLE_NAME', 'FUNCTION_NAME','COUNTROWSBEFORE', 'COUNTROWSAFTER', 'ERRORCODE', 'ERRORMESSAGE']})

In [21]:
log['PIPELINENAME'] = 'xxx'

In [22]:
log

{'PIPELINENAME': 'xxx', 'PIPELINERUNID': None, 'TRIGGERTYPE': None, 'TABLE_NAME': None, 'FUNCTION_NAME': None, 'COUNTROWSBEFORE': None, 'COUNTROWSAFTER': None, 'ERRORCODE': None, 'ERRORMESSAGE': None}

In [23]:
log['new']='yyy'

KeyError: 'Cannot add new key: new'

In [48]:
class logger:
    def __init__(self, **kwargs):
        self._data = kwargs

    def __getitem__(self, key):
        return self._data[key]

    def __setitem__(self, key, value):
        if key not in self._data:
            raise KeyError(f"Cannot add new key: {key}")
        self._data[key] = value

    def __delitem__(self, key):
        raise KeyError(f"Cannot delete key: {key}")

    def __iter__(self):
        return iter(self._data)

    def __len__(self):
        return len(self._data)

    def __repr__(self):
        return repr(self._data)

class AuditLog:
    
    def __init__(self, columns: Union[List[str], Tuple[str, ...]], WS_ID: str, TABLE_NAME_to_check:str, AUDIT_TABLE_NAME:str, LH_ID_to_check: str, LH_ID_audit: str = None, schema: str = None):
        '''
        - if `LH_ID_audit` is not given, it is  LH_ID_to_check automatically, i.e. audit table is in the same lakehouse as that of
        - if using lakehouse with Schema, please provide `schema` parameter
        '''
        self.WS_ID = WS_ID
        self.TABLE_NAME_to_check = TABLE_NAME_to_check
        self.AUDIT_TABLE_NAME = AUDIT_TABLE_NAME
        self.LH_ID_to_check = LH_ID_to_check
        self.LH_ID_audit = LH_ID_audit if LH_ID_audit else LH_ID_to_check
        self.schema = schema
        self.fixColumns = {'STARTTIME','ENDTIME','AUDITKEY','STATUS_ACTIVITY'}
        self.columns = tuple(set(columns).union(self.fixColumns))
        
        if self.schema:    
            self.PATH_TO_AUDIT_TABLE = f'abfss://{self.WS_ID}@onelake.dfs.fabric.microsoft.com/{self.LH_ID_audit}/Tables/{self.schema}/{self.AUDIT_TABLE_NAME}'
            self.PATH_TO_CHECKED_TABLE = f'abfss://{self.WS_ID}@onelake.dfs.fabric.microsoft.com/{self.LH_ID_to_check}/Tables/{self.schema}/{self.TABLE_NAME_to_check}'
        else:
            self.PATH_TO_AUDIT_TABLE = f'abfss://{self.WS_ID}@onelake.dfs.fabric.microsoft.com/{self.LH_ID_audit}/Tables/{self.AUDIT_TABLE_NAME}'
            self.PATH_TO_CHECKED_TABLE = f'abfss://{self.WS_ID}@onelake.dfs.fabric.microsoft.com/{self.LH_ID_to_check}/Tables/{self.TABLE_NAME_to_check}'
    
        self.log = logger(**{column: None for column in self.columns})
        self.log['STARTTIME'] = datetime.now() + timedelta(hours=7)
        self.log['STATUS_ACTIVITY'] = 'Not start'

    def setKey(self, initConfig: dict[str, Any]):
        assert set(initConfig.keys()).issubset(set(self.columns).difference()), f'initConfig must have the columns in {self.columns}'
        for column in initConfig:
            self.log[column] = initConfig[column]
        
    def initialDetail(self, initConfig: dict[str, Any]):
        self.setKey(initConfig)

    def getKey(self):
        return self.columns
    
    def getLog(self):
        return self.log
        
    def __str__(self):
        out = ''
        for key in self.columns:
            out += f'{key}: {self.log[key]}\n'
        return out
    
    def __repr__(self):
        return str(self.log)


class AuditLog_SPC(AuditLog):
    
    def __init__(self, WS_ID: str, TABLE_NAME_to_check:str, AUDIT_TABLE_NAME:str, LH_ID_to_check: str, LH_ID_audit: str = None, schema: str = None):
        '''
        - if `LH_ID_audit` is not given, it is  LH_ID_to_check automatically, i.e. audit table is in the same lakehouse as that of
        - if using lakehouse with Schema, please provide `schema` parameter
        '''
        super().__init__(['PIPELINENAME', 'PIPELINERUNID', 'TRIGGERTYPE', 'TABLE_NAME', 'FUNCTION_NAME','COUNTROWSBEFORE', 'COUNTROWSAFTER', 'ERRORCODE', 'ERRORMESSAGE'] ,WS_ID, TABLE_NAME_to_check, AUDIT_TABLE_NAME, LH_ID_to_check, LH_ID_audit, schema)

    def initialDetail(self,  pipelineName: str, pipelineId, TriggerType, TableName, functionName, ):
        super().initialDetail({
            'PIPELINENAME': pipelineName, 
            'PIPELINERUNID': pipelineId, 
            'TRIGGERTYPE': TriggerType, 
            'TABLE_NAME': TableName, 
            'FUNCTION_NAME': functionName
        })

{'STATUS_ACTIVITY': 'Not start', 'TRIGGERTYPE': 'manual', 'ERRORCODE': None, 'STARTTIME': datetime.datetime(2025, 1, 18, 20, 19, 37, 815088), 'ENDTIME': None, 'PIPELINERUNID': '123', 'TABLE_NAME': 'factTest', 'ERRORMESSAGE': None, 'PIPELINENAME': 'testPL', 'AUDITKEY': None, 'COUNTROWSBEFORE': None, 'COUNTROWSAFTER': None, 'FUNCTION_NAME': 'testFunction'}

In [51]:
print(ad)

STATUS_ACTIVITY: Not start
TRIGGERTYPE: manual
ERRORCODE: None
STARTTIME: 2025-01-18 20:19:37.815088
ENDTIME: None
PIPELINERUNID: 123
TABLE_NAME: factTest
ERRORMESSAGE: None
PIPELINENAME: testPL
AUDITKEY: None
COUNTROWSBEFORE: None
COUNTROWSAFTER: None
FUNCTION_NAME: testFunction



In [79]:
import numpy as np
from pyspark.sql.functions import col, when, concat, lit, format_string,sum, upper, substring, expr, current_date, current_timestamp,to_timestamp,concat_ws, isnull, date_format, asc, trim, trunc, date_sub, year,coalesce, count, countDistinct, min, max
from pyspark.sql.types import IntegerType, DecimalType, StringType, LongType, TimestampType, StructType, StructField, DoubleType, FloatType
import pandas as pd
from pyspark.sql import SparkSession
from datetime import datetime, timedelta
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor
from typing import Union, List, Tuple, Any
# from tabulate import tabulate

spark = SparkSession.builder\
        .appName("utils")\
        .getOrCreate()

class logger:
    def __init__(self, **kwargs):
        self._data = kwargs

    def __getitem__(self, key):
        return self._data[key]

    def __setitem__(self, key, value):
        if key not in self._data:
            raise KeyError(f"Cannot add new key: {key}")
        self._data[key] = value

    def __delitem__(self, key):
        raise KeyError(f"Cannot delete key: {key}")

    def __iter__(self):
        return iter(self._data)

    def __len__(self):
        return len(self._data)

    def __repr__(self):
        return repr(self._data)
    
# class Audit:
#     def __init__(self):
#         raise NotImplementedError('This is an abstract class')
    
#     def setKeys(self, initConfig: dict[str, Any]):
#         assert set(initConfig.keys()).issubset(set(self.columns).difference()), f'initConfig must have the columns in {self.columns}'
#         for column in initConfig:
#             self.log[column] = initConfig[column]

        

class AuditLog_Fusion:
    
    def __init__(self, columns: Union[List[str], Tuple[str, ...]], WS_ID: str, TABLE_NAME_to_check:str, AUDIT_TABLE_NAME:str, LH_ID_to_check: str, LH_ID_audit: str = None, schema: str = None):
        '''
        - if `LH_ID_audit` is not given, it is  LH_ID_to_check automatically, i.e. audit table is in the same lakehouse as that of
        - if using lakehouse with Schema, please provide `schema` parameter
        '''
        self.WS_ID = WS_ID
        self.TABLE_NAME_to_check = TABLE_NAME_to_check
        self.AUDIT_TABLE_NAME = AUDIT_TABLE_NAME
        self.LH_ID_to_check = LH_ID_to_check
        self.LH_ID_audit = LH_ID_audit if LH_ID_audit else LH_ID_to_check
        self.schema = schema
        self.fixColumns = {'STARTTIME','ENDTIME','AUDITKEY','STATUS_ACTIVITY'}
        self.columns = tuple(set(columns).union(self.fixColumns))
        
        if self.schema:    
            self.PATH_TO_AUDIT_TABLE = f'abfss://{self.WS_ID}@onelake.dfs.fabric.microsoft.com/{self.LH_ID_audit}/Tables/{self.schema}/{self.AUDIT_TABLE_NAME}'
            self.PATH_TO_CHECKED_TABLE = f'abfss://{self.WS_ID}@onelake.dfs.fabric.microsoft.com/{self.LH_ID_to_check}/Tables/{self.schema}/{self.TABLE_NAME_to_check}'
        else:
            self.PATH_TO_AUDIT_TABLE = f'abfss://{self.WS_ID}@onelake.dfs.fabric.microsoft.com/{self.LH_ID_audit}/Tables/{self.AUDIT_TABLE_NAME}'
            self.PATH_TO_CHECKED_TABLE = f'abfss://{self.WS_ID}@onelake.dfs.fabric.microsoft.com/{self.LH_ID_to_check}/Tables/{self.TABLE_NAME_to_check}'
    
        self.log = logger(**{column: None for column in self.columns})
        self.log['STARTTIME'] = datetime.now()
        self.log['STATUS_ACTIVITY'] = 'Not start'

    def setKeys(self, initConfig: dict[str, Any]):
        assert set(initConfig.keys()).issubset(set(self.columns).difference()), f'initConfig must have the columns in {self.columns}'
        for column in initConfig:
            self.log[column] = initConfig[column]

    def setKey(self, key: str, value: Any):
        assert key in self.columns, f'key must be in {self.columns}'
        self.log[key] = value
        
    def initialDetail(self, initConfig: dict[str, Any]):
        self.setKeys(initConfig)

    def getKey(self):
        return self.columns
    
    def getLog(self):
        return self.log
        
    def __str__(self):
        out = ''
        for key in self.columns:
            out += f'{key}: {self.log[key]}\n'
        return out
    
    def __repr__(self):
        return str(self.log)
    
    def endSuccess(self):
        self.log['STATUS_ACTIVITY'] = 'Fail'
        self._endAuditLog()
        print(self)
        
    def endFail(self, errorCode: str, errorMessage: str):
        self.log['STATUS_ACTIVITY'] = 'Fail'
        self.log['ERRORCODE'] = errorCode
        self.log['ERRORMESSAGE'] = errorMessage
        self._endAuditLog()
        print(self)

    def _endAuditLog(self):
        # write to audit table
        self.log['ENDTIME'] = datetime.now()
        df = spark.createDataFrame([self.log])
        df.write.mode('append').save(self.PATH_TO_AUDIT_TABLE)

    def getAuditLogTable(self):
        return spark.read.load(self.PATH_TO_AUDIT_TABLE)
    
    def countBefore(self, df):
        self.log['COUNTROWSBEFORE'] = df.count()

    def countAfter(self, df):
        self.log['COUNTROWSAFTER'] = df.count()

    def getAllPath(self):
        return {'PATH_TO_AUDIT_TABLE':self.PATH_TO_AUDIT_TABLE, 'PATH_TO_CHECKED_TABLE':self.PATH_TO_CHECKED_TABLE}


class AuditLog_SPC(AuditLog_Fusion):
    
    def __init__(self, WS_ID: str, TABLE_NAME_to_check:str, AUDIT_TABLE_NAME:str, LH_ID_to_check: str, LH_ID_audit: str = None, schema: str = None):
        '''
        - if `LH_ID_audit` is not given, it is  LH_ID_to_check automatically, i.e. audit table is in the same lakehouse as that of
        - if using lakehouse with Schema, please provide `schema` parameter
        '''
        super().__init__(['PIPELINENAME', 'PIPELINERUNID', 'TRIGGERTYPE', 'TABLE_NAME', 'FUNCTION_NAME','COUNTROWSBEFORE', 'COUNTROWSAFTER', 'ERRORCODE', 'ERRORMESSAGE'] ,WS_ID, TABLE_NAME_to_check, AUDIT_TABLE_NAME, LH_ID_to_check, LH_ID_audit, schema)

    def initialDetail(self,  pipelineName: str, pipelineId: str, TriggerType: str, TableName: str, functionName: str):
        super().initialDetail({
            'PIPELINENAME': pipelineName, 
            'PIPELINERUNID': pipelineId, 
            'TRIGGERTYPE': TriggerType, 
            'TABLE_NAME': TableName, 
            'FUNCTION_NAME': functionName
        })


In [80]:
ad = AuditLog_SPC(WS_ID = 'SPC_UAT', TABLE_NAME_to_check = 'factTest', AUDIT_TABLE_NAME='auditTable', LH_ID_to_check='SilverLH', LH_ID_audit='AuditLH')
ad.initialDetail(pipelineName = 'testPL', pipelineId = '123', TriggerType = 'manual', TableName = 'factTest', functionName = 'testFunction')
print(ad)

raise FileExistsError('Create you audit table first')

STATUS_ACTIVITY: Not start
TRIGGERTYPE: manual
ERRORCODE: None
STARTTIME: 2025-01-18 13:40:43.660532
ENDTIME: None
PIPELINERUNID: 123
TABLE_NAME: factTest
ERRORMESSAGE: None
PIPELINENAME: testPL
AUDITKEY: None
COUNTROWSBEFORE: None
COUNTROWSAFTER: None
FUNCTION_NAME: testFunction



In [82]:
demoDf = spark.createDataFrame([{'a':1, 'b':2}, {'a':3, 'b':4}])
demoDf.count()

DataFrame[a: bigint, b: bigint]

In [77]:
notebookutils.fs.exists(ad.PATH_TO_AUDIT_TABLE)

STATUS_ACTIVITY: Not start
TRIGGERTYPE: manual
ERRORCODE: None
STARTTIME: 2025-01-18 13:39:16.665795
ENDTIME: None
PIPELINERUNID: 123
TABLE_NAME: factTest
ERRORMESSAGE: None
PIPELINENAME: testPL
AUDITKEY: None
COUNTROWSBEFORE: None
COUNTROWSAFTER: None
FUNCTION_NAME: testFunction



In [3]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# ข้อมูลการตั้งค่าของผู้ส่ง
gmail_user = 'phaphontee.yam@student.mahidol.edu'         # ใส่ที่อยู่อีเมลของคุณ
app_password = 'rziq gbow xoer qlnv'          # ใส่ App Password ที่สร้างไว้

# ข้อมูลของผู้รับและเนื้อหาอีเมล
to = 'phaphontee@fusionsol.com'          # ใส่อีเมลผู้รับ
subject = 'ทดสอบการส่งอีเมลด้วย Python'   # ใส่หัวเรื่อง
body = 'สวัสดีครับ นี่เป็นอีเมลทดสอบที่ส่งด้วย Python!'  # ใส่เนื้อหาของอีเมล

# การสร้างอีเมล
msg = MIMEMultipart()
msg['From'] = gmail_user
msg['To'] = to
msg['Subject'] = subject
msg.attach(MIMEText(body, 'plain'))

try:
    # การเชื่อมต่อกับ SMTP ของ Gmail
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()  # เปิดการเข้ารหัส TLS
    server.login(gmail_user, app_password)  # เข้าสู่ระบบด้วยอีเมลและรหัสผ่านแอป
    text = msg.as_string()

    # ส่งอีเมล
    server.sendmail(gmail_user, to, text)
    print("อีเมลถูกส่งสำเร็จ!")

except Exception as e:
    print(f"เกิดข้อผิดพลาด: {e}")

finally:
    # ปิดการเชื่อมต่อ
    server.quit()



อีเมลถูกส่งสำเร็จ!


In [11]:
import numpy as np
from pyspark.sql.functions import col, when, concat, lit, format_string,sum, upper, substring, expr, current_date, current_timestamp,to_timestamp,concat_ws, isnull, date_format, asc, trim, trunc, date_sub, year,coalesce, count, countDistinct, min, max
from pyspark.sql.types import IntegerType, DecimalType, StringType, LongType, TimestampType, StructType, StructField, DoubleType, FloatType
import pandas as pd
from pyspark.sql import SparkSession
from datetime import datetime, timedelta
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor
from typing import Union, List, Tuple, Any
# from tabulate import tabulate

spark = SparkSession.builder\
        .appName("test")\
        .getOrCreate()

In [12]:
from src.etl.factCPU import CashPickUp_SAP

In [14]:
import pandas as pd

In [ ]:
filename = "1106070_25_03_13.XLSx"
df = pd.read_excel(filename,skiprows=4).fillna("").astype("str").sum(axis=1).str.split("\t",expand=True)
df_col = df.iloc[0].str.replace('.','').str.replace('G/L','G_L')
df = df.iloc[2:]
df.columns = df_col.str.strip().apply(lambda x: '-' if len(x) == 0 else x)
df = df.iloc[:-2]
df = df.drop(columns='-')
df = df[df['Text'].str.contains('CPICK')]
df['AssignmentDate'] = df['Assignment'].str[-8:]
df['DocDate'] = pd.to_datetime(df['Doc Date'].str.replace(".",""),format="%m%d%Y").dt.strftime('%Y%m%d')
df['POSNumber'] = df['Assignment'].str[:5]
df['Amount in doc curr'] = df['Amount in doc curr'].str.strip()
df['DocumentNo'] = df['DocumentNo'].str.strip()
df['AmountCURR'] = df['Amount in doc curr'].str.extract(r'([^\.]+)\..*',expand=False).str.replace(r'[\.,]',"",regex=True)
df['FileName'] = filename
df['ETL_Date'] = datetime.now()
df['YYYYMM_DocDate'] = pd.to_datetime(df['Doc Date'],format='%d.%m.%Y').dt.strftime('%Y%m')

df

,St,Reference,BP,G_L,Cost,Assignment,DocumentNo,Ty,Doc Date,PK,...,Clrng doc,Materia,Text,AssignmentDate,DocDate,POSNumber,AmountCURR,FileName,ETL_Date,YYYYMM_DocDate


In [152]:
filename = "1000020_25_03_13.XLSx"
df = pd.read_excel(filename,skiprows=4).fillna("").astype("str").sum(axis=1).str.split("\t",expand=True)
df_col = df.iloc[0].str.replace('.','').str.replace('G/L','G_L')
df = df.iloc[2:]
df.columns = df_col.str.strip().apply(lambda x: '-' if len(x) == 0 else x)
df = df.iloc[:-2]
df = df.drop(columns='-')
df = df[df['G_L'].str.len()>=1]
df['AssignmentDate'] = df['Assignment'].str[-8:]
df['DocDate'] = pd.to_datetime(df['Doc Date'].str.replace(".",""),format="%m%d%Y").dt.strftime('%Y%m%d')
df['POSNumber'] = df['Assignment'].str[:5]
df['Amount in doc curr'] = df['Amount in doc curr'].str.strip()
df['DocumentNo'] = df['DocumentNo'].str.strip()
df['AmountCURR'] = df['Amount in doc curr'].str.extract(r'([^\.]+)\..*',expand=False).str.replace(r'[\.,]',"",regex=True)
df['FileName'] = filename
df['ETL_Date'] = datetime.now()
df['YYYYMM_DocDate'] = pd.to_datetime(df['Doc Date'],format='%d.%m.%Y').dt.strftime('%Y%m')

df

,St,Reference,BP,G_L,Cost,Assignment,DocumentNo,Ty,Doc Date,PK,...,Clrng doc,Materia,Text,AssignmentDate,DocDate,POSNumber,AmountCURR,FileName,ETL_Date,YYYYMM_DocDate
2,,,0184,1000020,,1001920250301,5106004555,GC,01.03.2025,50,...,,,01.03.2025ปป.ยอดเงินดรอป10019,20250301,20250103,10019,-991,1000020_25_03_13.XLSx,2025-03-31 20:02:23.208218,202503
3,,,0035,1000020,,1003220250301,5106004559,GC,01.03.2025,50,...,,,01.03.2025ปป.ยอดเงินดรอป10032,20250301,20250103,10032,-50,1000020_25_03_13.XLSx,2025-03-31 20:02:23.208218,202503
4,,,0014,1000020,,1004020250301,5106004561,GC,01.03.2025,40,...,,,01.03.2025ปป.ยอดเงินดรอป10040,20250301,20250103,10040,224167,1000020_25_03_13.XLSx,2025-03-31 20:02:23.208218,202503
5,,,0059,1000020,,1008520250301,5106004565,GC,01.03.2025,40,...,,,01.03.2025ปป.ยอดเงินดรอป10085,20250301,20250103,10085,136861,1000020_25_03_13.XLSx,2025-03-31 20:02:23.208218,202503
6,,,0257,1000020,,1010420250301,5106004569,GC,01.03.2025,50,...,,,01.03.2025ปป.ยอดเงินดรอป10104,20250301,20250103,10104,-2584,1000020_25_03_13.XLSx,2025-03-31 20:02:23.208218,202503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3896,,22220,0031,1000020,,2222020250312,5302021396,G3,12.03.2025,40,...,,,CDRP1 เงินสด Drop ลงเซฟ,20250312,20251203,22220,65643,1000020_25_03_13.XLSx,2025-03-31 20:02:23.208218,202503
3897,,22413,0168,1000020,,2241320250312,5302021398,G3,12.03.2025,40,...,,,CDRP1 เงินสด Drop ลงเซฟ,20250312,20251203,22413,34419,1000020_25_03_13.XLSx,2025-03-31 20:02:23.208218,202503
3898,,22430,0207,1000020,,2243020250312,5302021399,G3,12.03.2025,40,...,,,CDRP1 เงินสด Drop ลงเซฟ,20250312,20251203,22430,71098,1000020_25_03_13.XLSx,2025-03-31 20:02:23.208218,202503
3899,,22442,0231,1000020,,2244220250312,5302021400,G3,12.03.2025,40,...,,,CDRP1 เงินสด Drop ลงเซฟ,20250312,20251203,22442,160493,1000020_25_03_13.XLSx,2025-03-31 20:02:23.208218,202503


In [150]:
df[['Doc Date', 'DocDate']]

,Doc Date,DocDate
2,01.03.2025,20250103
3,01.03.2025,20250103
4,01.03.2025,20250103
5,01.03.2025,20250103
6,01.03.2025,20250103
...,...,...
3896,12.03.2025,20251203
3897,12.03.2025,20251203
3898,12.03.2025,20251203
3899,12.03.2025,20251203
